# Projet Neo4J - Yelp
*Réalisé par:*
* Issa Abakar (22112414)
* Ahmed Amine Ghorbel (22214386)

In [ ]:
import json
import csv
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
users_json_path = "/content/drive/MyDrive/M2/BigData/projet_neo4j/yelp_user.json"
restos_json_path = "/content/drive/MyDrive/M2/BigData/projet_neo4j/yelp_restaurants.json"
reviews_json_path = "/content/drive/MyDrive/M2/BigData/projet_neo4j/yelp_review.json"

### Data loading

In [ ]:
# chargement de données
with open(restos_json_path) as f:
    data_resto = json.load(f)

with open(reviews_json_path) as f:
    data_reviews = json.load(f)

with open(users_json_path) as f:
    data_user = json.load(f)

In [ ]:
# affichage des json
data_resto[36]

{'business_id': 'fCFUlCHG4spZ3O0vbqnYjA',
 'name': "Pat's Pizzeria",
 'address': '2008 W Newport Pike',
 'city': 'Wilmington',
 'state': 'DE',
 'postal_code': '19804',
 'latitude': 39.715031,
 'longitude': -75.634876,
 'stars': 2.0,
 'review_count': 5,
 'is_open': 1,
 'attributes': {'BusinessAcceptsCreditCards': 'True',
  'NoiseLevel': "u'average'",
  'OutdoorSeating': 'False',
  'Caters': 'True',
  'BikeParking': 'False'},
 'categories': 'Pizza, Restaurants',
 'hours': None}

In [ ]:
data_reviews[0]

{'review_id': '2xlNGuN8naQTHO3_xY0lGw',
 'user_id': '8rrmvJ7W3ujplXuOhjE2Hg',
 'business_id': 'yDiWYpuzRxcTgSi_O4JZpQ',
 'stars': 5.0,
 'useful': 0,
 'funny': 0,
 'cool': 0,
 'text': "This is the place for authentic Indian street food. I've been here multiple times ordering different things ranging from bhel puri, pav bhaji, vada pav, deconstructed samosa and aloo tikki, dabeli, kulfi and more. Everything was made exceptionally well.\n\nI regret not having found this place sooner.",
 'date': '2017-03-27 13:27:52'}

In [ ]:
data_user[0]

{'user_id': 'XLs_PhrJ7Qwn_RfgMM7Djw',
 'name': 'Weili',
 'review_count': 90,
 'yelping_since': '2009-07-12 14:34:54',
 'useful': 71,
 'funny': 21,
 'cool': 19,
 'elite': '',
 'friends': [],
 'fans': 1,
 'average_stars': 3.62,
 'compliment_hot': 0,
 'compliment_more': 0,
 'compliment_profile': 0,
 'compliment_cute': 0,
 'compliment_list': 0,
 'compliment_note': 1,
 'compliment_plain': 3,
 'compliment_cool': 0,
 'compliment_funny': 0,
 'compliment_writer': 0,
 'compliment_photos': 0}

### Data Transformation

In [ ]:
import pandas as pd


# Fontion permettant de convertir les données json en csv 

def to_csv(users_cols,reviews_cols,restos_cols):
  
  # to_df
  users_df = pd.DataFrame(data_user)
  reviews_df = pd.DataFrame(data_reviews)
  restos_df = pd.json_normalize(data_resto)

  # prendre uniquement les cols qui comptent
  users_df = users_df[users_cols]
  reviews_df = reviews_df[reviews_cols]
  restos_df = restos_df[restos_cols]

  # to_csv
  users_csv = users_df.to_csv("users_csv.csv",escapechar='\\', quotechar='"', quoting=csv.QUOTE_ALL)
  reviews_csv = reviews_df.to_csv("reviews_csv.csv",escapechar='\\', quotechar='"', quoting=csv.QUOTE_ALL)
  restos_csv = restos_df.to_csv("restos_csv.csv",escapechar='\\', quotechar='"', quoting=csv.QUOTE_ALL)

  return users_csv,reviews_csv,restos_csv


In [ ]:
# Les colonnes qu'on aura besoin pour le calcul des score
restos_cols = ["business_id","name","address","city","state","stars","review_count","attributes.RestaurantsPriceRange2","attributes.Ambience","categories"]
reviews_cols = ['review_id','user_id','business_id','useful','cool','text','date','stars']
users_cols = ['user_id','name','review_count','friends','cool','elite','fans']

In [ ]:
users_csv, reviews_csv,restos_csv = to_csv(users_cols,reviews_cols,restos_cols)

In [ ]:
users_csv_path = "/content/users_csv.csv"
review_csv_path = "/content/reviews_csv.csv"
restos_csv_path = "/content/restos_csv.csv"

In [ ]:
# Fonction permettant d'afficher un fichier csv sur n lignes
def print_csv(path,n):
  # opening the CSV file
  with open(path, mode ='r')as file:
    # reading the CSV file
    users_csv = csv.reader(file)
    # displaying the contents of the CSV file
    c = 0
    for lines in users_csv:
      if c < n:
        print(lines)
        c+=1

In [ ]:
# csv user 
print_csv(users_csv_path,20)

['', 'user_id', 'name', 'review_count', 'friends', 'cool', 'elite', 'fans']
['0', 'XLs_PhrJ7Qwn_RfgMM7Djw', 'Weili', '90', '[]', '19', '', '1']
['1', 'IpLRJY4CP3fXtlEd8Y4GFQ', 'Robyn', '518', "['y3Wtx1pOvTiqsJRkjceXiw', '_BcWyKQL16ndpBdggh2kNA', 'lRRuTimITgwzoXLIM3g9qw', 'z02XKjGqJMONH59_lUHlTQ', '47K1MpOBhigOvjuBcDbm7Q', '2-RlTk9dJNpj85MR2B46Cg', '_hPjqzQDiyw1-4U_IAETpg', 'dhLizr4a2oydrv15y56r0A', 'rkxGrjk2WeAT5ZMfChT3kw', 'pbCb0Tz0KQ0kkgaxzhWhoA', 'Ed0GuUa_1UjbSk8qaQsHlA', '_LNCGYNoGKxyVUag3EG0-g', 'B9iPTih0dmpZn6fRyZnLLg', 'gTrKXmAx1ytdvh1xutIlfA', 'u1gUbwyeBe8mtV97hZyImg', 'UiRx8jyS6H957ItwFsP2nQ', 'IPxJLWpKptzpTWlfjFiuHA', 'srETv2HpH9fQ-7wCvCAQng', 'cUB2WhICFuSe7DwXAIoo0g', 'vU9_lZqJQIR8btUYw7fqxA', 'pLZIZXIebqEcrX7IVKoJRg', 'IWxNILja-WvurTuJWJ4skA', '63hpZgj7peTKizDs4zeKZA']", '348', '2009,2010,2011', '35']
['2', '0juMoWXC7z4c7LgQP2s3sA', 'JR', '199', "['46Wi08T5D3phf3SMgi7U3w', 'HUxktqjf5kJm0tIHTCCwsQ', 'X-8WUdJ1hYidYXQygzTEBw', 'sGaoPWB_p1aOm5z9cnV3Aw']", '131', '2013,2014,2015

In [ ]:
# csv reviews
print_csv(review_csv_path,20)

['', 'review_id', 'user_id', 'business_id', 'useful', 'cool', 'text', 'date', 'stars']
['0', '2xlNGuN8naQTHO3_xY0lGw', '8rrmvJ7W3ujplXuOhjE2Hg', 'yDiWYpuzRxcTgSi_O4JZpQ', '0', '0', "This is the place for authentic Indian street food. I've been here multiple times ordering different things ranging from bhel puri, pav bhaji, vada pav, deconstructed samosa and aloo tikki, dabeli, kulfi and more. Everything was made exceptionally well.\n\nI regret not having found this place sooner.", '2017-03-27 13:27:52', '5.0']
['1', '_Dkml_CfMMb_wWeGpFtczg', 'TRFTDMJegE6DituEKBpEKQ', 'W6S482dRQ_9nCcLjfv8VNg', '0', '0', "Food was great. But very disappointed with the hospitality. We  entered the restaurant at 9:15 pm but was denied dine in and was told that though the  restaurant closes at 10 pm they won't let us dine in and can give only to go. They saw that there were two pregnant women and a kid with us and I explained that our home is a hour drive from there and yet they denied for no reason. I had 

In [ ]:
# csv restos
print_csv(restos_csv_path,20)

['', 'business_id', 'name', 'address', 'city', 'state', 'stars', 'review_count', 'attributes.RestaurantsPriceRange2', 'attributes.Ambience', 'categories']
['0', 'Dy91wdWkwtI_qgjAIZ0Niw', 'Taqueria Los Primos', '3322 Old Capitol Trl', 'Wilmington', 'DE', '4.0', '23', '1', "{u'divey': True, u'hipster': False, u'casual': None, u'touristy': False, u'trendy': False, u'intimate': False, u'romantic': False, u'classy': False, u'upscale': False}", 'Mexican, Restaurants']
['1', '_Oksx3ALgKLRVxeAR8l_qA', 'V-Trap Kitchen and Lounge', '607 N Lincoln St', 'Wilmington', 'DE', '5.0', '13', '2', "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': True, 'casual': False}", 'Restaurants, Mexican, Arts & Entertainment, Event Planning & Services, Italian, Gluten-Free, Nightlife, Lounges, Vegan, Bars, Caterers, Vegetarian']
['2', '_NhT0A3LUEd75L_cEXf2_A', "Season's Pizza", '1721 W Gilpin Dr', 'Wilmington', 'DE', '3.0', '7',

In [ ]:
# Create ID DICT 
# Associé à chaque ID un int 
# Le fait de transformer les ID en valeur entiére nous permettra de bien faire la différence entre les ids
# Et faire les tests facilement 
ID_DICT = {}
ID_DICT_REVIEW = {}
ID_DICT_BUSINESS = {}

def create_id_dict(users,reviews,business):
  # opening the CSV file
  with open(users, mode ='r') as users_file, open(reviews, mode ='r') as reviews_file, open(business, mode ='r') as business_file  :
    # reading the CSV file
    users_csv = csv.reader(users_file)
    reviews_csv = csv.reader(reviews_file)
    business_csv = csv.reader(business_file)
    # displaying the contents of the CSV file
    c = 0

    for users_lines in users_csv:
      # si c'est pas les headers
      if c > 0:
        ID_DICT[users_lines[1]] = users_lines[0]
      c+=1

    c = 0
    for reviews_lines in reviews_csv:
      # si c'est pas les headers
      if c > 0:
        ID_DICT_REVIEW[reviews_lines[1]] = reviews_lines[0]
      c+=1
    
    c = 0
    for business_lines in business_csv:
      # si c'est pas les headers
      if c > 0:
        ID_DICT_BUSINESS[business_lines[1]] = business_lines[0]
      c+=1

In [ ]:
create_id_dict(users_csv_path,review_csv_path,restos_csv_path)

In [ ]:
# On peut voir que chaque ID initial est associé à une valeur entiére 
print(len(list(ID_DICT.items())))
list(ID_DICT.items())[:10]

23082


[('XLs_PhrJ7Qwn_RfgMM7Djw', '0'),
 ('IpLRJY4CP3fXtlEd8Y4GFQ', '1'),
 ('0juMoWXC7z4c7LgQP2s3sA', '2'),
 ('FlhZYUEhfpfm0Ynxacftwg', '3'),
 ('PadmV2GEoA6mWpQUpPh7Ig', '4'),
 ('1-gnHNzZNbn8lFwYRF_CtA', '5'),
 ('UJEjJgi1xpmLR4b3J8fn2Q', '6'),
 ('7LmJ87EijrHcoYRu9WgoHg', '7'),
 ('FH14lvoIAKhva9KVgvzKJw', '8'),
 ('kLm3B6LeboEZWX_4otpILA', '9')]

In [ ]:
print(len(list(ID_DICT_REVIEW.items())))
list(ID_DICT_REVIEW.items())[:10]

49150


[('2xlNGuN8naQTHO3_xY0lGw', '0'),
 ('_Dkml_CfMMb_wWeGpFtczg', '1'),
 ('EmLTCw2p9BhOwnzHFXgGaQ', '2'),
 ('PlYifdeyiixRZkymbUNqVQ', '3'),
 ('PtUd264rjqWuEzL89nf64w', '4'),
 ('jh63cU95B-Kjgyev_a_4oQ', '5'),
 ('2DUxwJgRW1Ds_EN763CiQQ', '6'),
 ('lV_V_FklhhmMaWRXAB_tyA', '7'),
 ('OaYClWZ3UyrAWo60tGCVsg', '8'),
 ('TM1PkkUDSMpb0ijfePRmBQ', '9')]

In [ ]:
print(len(list(ID_DICT_BUSINESS.items())))
list(ID_DICT_BUSINESS.items())[:10]

961


[('Dy91wdWkwtI_qgjAIZ0Niw', '0'),
 ('_Oksx3ALgKLRVxeAR8l_qA', '1'),
 ('_NhT0A3LUEd75L_cEXf2_A', '2'),
 ('A_-Vv5oKZNqxsV69_7E2KQ', '3'),
 ('JNH-hcySeuGmtI_t8dG3Dw', '4'),
 ('ekBvLA43oEWEswUXZI0bYQ', '5'),
 ('5R6YO6K9PkthkTbXqV9slQ', '6'),
 ('iXemjTj5TOByT1-cNTSaDw', '7'),
 ('5ODCsQbRdsFIWlypGF3iUw', '8'),
 ('s-7Q1GAQmB-Qsm_UwviHVA', '9')]

In [ ]:
# Création de la relation FRIEND
def create_friends_relations(users_data):

  friends_df = pd.DataFrame(columns=[":START_ID",":END_ID"])
  x = 0
  for i in range(len(users_data)):
    user = users_data[i]
    friends = user['friends']
    for friend_id in friends:
      friends_df.loc[x] = [ID_DICT[user['user_id']],ID_DICT[friend_id]]
      x+=1

  return friends_df.to_csv("friends.csv",escapechar='\\', quotechar='"', quoting=csv.QUOTE_ALL,mode="w",index=False)

In [ ]:
friends = create_friends_relations(data_user)

In [ ]:
friends_path = "/content/friends.csv"
print_csv(friends_path,30)

[':START_ID', ':END_ID']
['1', '41']
['1', '32']
['1', '362']
['1', '219']
['1', '12338']
['1', '4329']
['1', '553']
['1', '1122']
['1', '16827']
['1', '1527']
['1', '600']
['1', '1590']
['1', '501']
['1', '6871']
['1', '492']
['1', '4134']
['1', '44']
['1', '830']
['1', '1200']
['1', '2015']
['1', '1956']
['1', '70']
['1', '329']
['2', '226']
['2', '14339']
['2', '96']
['2', '1316']
['3', '4134']
['3', '84']


In [ ]:
# Bien qu'on a déjà tranformé les fichiers json en csv, cette fonction permettra de prendre en compte
# Les nouvaux ID determinés ci - haut. On aura une affichage plus claire
# C'est les fichiers csv générés par cette fonction qui seront importé dans Neo4j. 

def transform_data(users_json_path,reviews_json_path,business_json_path):

  # loading json data
  with open(reviews_json_path) as f_review, open(users_json_path) as f_users, open(business_json_path) as f_business:
    reviews_data = json.load(f_review)
    users_data = json.load(f_users)
    business_data = json.load(f_business)


  # User data transforming
  users_df = pd.DataFrame(columns=users_cols)
  x = 0
  for u in range(len(users_data)):
      user_id = users_data[u]['user_id']
      friends = users_data[u]['friends']
      friends_id = [int(ID_DICT[friend_id]) for friend_id in friends]
      if len(user_id) > 1 :
        line = [ID_DICT[user_id],users_data[u]['name'],users_data[u]['review_count'],friends_id,users_data[u]['cool'],users_data[u]['elite'],users_data[u]['fans']]
      else :
        line = [int(user_id),users_data[u]['name'],users_data[u]['review_count'],friends_id,users_data[u]['cool'],users_data[u]['elite'],users_data[u]['fans']]
      users_df.loc[x] = line
      x+=1
    
  # review data transforming
  x = 0
  for u in range(len(reviews_data)):
      reviews_data[u]['user_id'] = ID_DICT[reviews_data[u]['user_id']]
      reviews_data[u]['review_id'] = ID_DICT_REVIEW[reviews_data[u]['review_id']]
      reviews_data[u]['business_id'] = ID_DICT_BUSINESS[reviews_data[u]['business_id']]
      x+=1
  reviews_df = pd.DataFrame(reviews_data,columns=reviews_cols)

  # review data transforming
  for b in range(len(business_data)):
    business_data[b]['business_id'] = ID_DICT_BUSINESS[business_data[b]['business_id']]
  business_df = pd.json_normalize(business_data)


  # loading data on csv files
  users_df.to_csv("new_users.csv",escapechar='\\', quotechar='"', quoting=csv.QUOTE_ALL,mode="w",index=False)
  reviews_df.to_csv("new_reviews.csv",escapechar='\\', quotechar='"', quoting=csv.QUOTE_ALL,mode="w",index=False)
  business_df.to_csv("new_business.csv",escapechar='\\', quotechar='"', quoting=csv.QUOTE_ALL,mode="w",index=False)


In [ ]:
transform_data(users_json_path,reviews_json_path,business_json_path)

In [ ]:
# affichage des csv aprés transformation
new_users_path = "/content/new_users.csv"
print_csv(new_users_path,20)

['user_id', 'name', 'review_count', 'friends', 'cool', 'elite', 'fans']
['0', 'Weili', '90', '[]', '19', '', '1']
['1', 'Robyn', '518', '[41, 32, 362, 219, 12338, 4329, 553, 1122, 16827, 1527, 600, 1590, 501, 6871, 492, 4134, 44, 830, 1200, 2015, 1956, 70, 329]', '348', '2009,2010,2011', '35']
['2', 'JR', '199', '[226, 14339, 96, 1316]', '131', '2013,2014,2015,2016', '9']
['3', 'Kate', '94', '[4134, 84, 1326, 821, 1320]', '80', '', '6']
['4', 'Monica', '695', '[14249, 66]', '861', '2010,2011,2012,2013,2014,2015', '104']
['5', 'Liz', '112', '[33, 4226, 12338]', '256', '2012,2013,2014', '36']
['6', 'Ken', '341', '[53, 442, 461, 96, 327, 5768, 361, 229, 270, 226, 239, 307, 227, 212, 834, 362, 162, 1267, 763, 279, 116, 21, 1235, 271, 54, 553, 433, 103, 1173, 460, 1200, 282, 32, 1399, 135, 1332, 1444, 580, 1175, 821, 1677, 773, 154, 351, 2332, 2171, 1708, 355, 2222, 4156, 29, 2678, 119, 147, 741, 1287, 4910, 202, 1366, 84, 41, 2015, 1039, 1169, 1316, 137, 124, 2524, 868, 21946, 1531, 1312, 

In [ ]:
# Creation de relations REVIEW_OF et WROTE 

def create_reviews_relations(reviews_json_path):
  
  #loading json data
  with open(reviews_json_path) as f:
    reviews_data = json.load(f)

  wrote_df = pd.DataFrame(columns=[":START_ID(User)",":END_ID(Review)"])
  review_of_df = pd.DataFrame(columns=[":START_ID(Review)",":END_ID(Business)"])

  for i in range(len(reviews_data)):
    review = reviews_data[i]
    user_id = review['user_id']
    review_id = review['review_id']
    business_id = review['business_id']
    
    wrote_df.loc[i] = [ID_DICT[user_id],ID_DICT_REVIEW[review_id]]
    review_of_df.loc[i] = [ID_DICT_REVIEW[review_id],ID_DICT_BUSINESS[business_id]]

  wrote_df.to_csv("wrote.csv",escapechar='\\', quotechar='"', quoting=csv.QUOTE_ALL,mode="w",index=False)
  review_of_df.to_csv("review_of.csv",escapechar='\\', quotechar='"', quoting=csv.QUOTE_ALL,mode="w",index=False)

In [ ]:
create_reviews_relations(reviews_json_path)

In [ ]:
wrote_path = "/content/wrote.csv"
review_of_path = "/content/review_of.csv"

In [ ]:
# Affichage de la relation WROTE
print_csv(wrote_path,30)

[':START_ID(User)', ':END_ID(Review)']
['2443', '0']
['3055', '1']
['3453', '2']
['268', '3']
['936', '4']
['4078', '5']
['415', '6']
['1045', '7']
['288', '8']
['3169', '9']
['3629', '10']
['1832', '11']
['4020', '12']
['3919', '13']
['1459', '14']
['1274', '15']
['763', '16']
['3134', '17']
['3169', '18']
['599', '19']
['396', '20']
['1943', '21']
['4040', '22']
['1908', '23']
['332', '24']
['1748', '25']
['1493', '26']
['2608', '27']
['3648', '28']


In [ ]:
# Affichage de la relation REVIEW_OF
print_csv(review_of_path,30)

[':START_ID(Review)', ':END_ID(Business)']
['0', '32']
['1', '67']
['2', '25']
['3', '54']
['4', '73']
['5', '49']
['6', '49']
['7', '12']
['8', '22']
['9', '45']
['10', '29']
['11', '23']
['12', '33']
['13', '33']
['14', '38']
['15', '3']
['16', '30']
['17', '34']
['18', '34']
['19', '66']
['20', '25']
['21', '0']
['22', '70']
['23', '66']
['24', '20']
['25', '57']
['26', '49']
['27', '29']
['28', '25']
